In [ ]:
import math as m
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.patches import Rectangle

from copy import deepcopy

In [ ]:
# Plate sizes
a, b = 80, 60
c, e = 20, 20
d, g = 40, 40

# Bound temperature
u0 = 300
u1 = 320

# Heat source
f0 = 60
beta = 0.1
x0, z0 = 9, 45

def f(x,z):
    return f0 * np.exp(-beta * ((x - x0)**2 + (z - z0)**2))

ka = 20

Nx, Nz = 80, 60 # Mesh sizes

eps = 0.1

In [ ]:
# Steps
hx, hz = a / Nx, b / Nz
tau = 0.25 * min(hx**2, hz**2)

Mx = int(c / hx)
Mz = int(e / hz)
Kx = int(d / hx)
Kz = int(g / hz)

x = np.linspace(0, a, Nx + 1)
z = np.linspace(0, b, Nz + 1)

In [ ]:
# First iteration

u = u0 * np.ones((Nx + 1, Nz + 1))
for i in range(Mx, Kx + 1):
    for j in range(Mz, Kz + 1):
        u[i,j] = u1

In [ ]:
# Triagonal solving

def tridiag(a, b, c, f):
    a, b, c, f = tuple(map(lambda k_list: list(map(float, k_list)), (a, b, c, f)))
    n = len(f)

    alpha = [-b[0] / c[0]] + [0] * (n - 1)
    beta = [f[0] / c[0]] + [0] * (n - 1)
    x = [0] * n

    for i in range(1, n):
        alpha[i] = -b[i] / (a[i] * alpha[i - 1] + c[i])
        beta[i] = (f[i] - a[i] * beta[i - 1]) / (a[i] * alpha[i - 1] + c[i])

    x[n - 1] = beta[n - 1]
    for i in range(n - 1, 0, -1):
        x[i - 1] = alpha[i - 1] * x[i] + beta[i - 1]

    return x

In [ ]:
m_long_x = -(2 + hx**2 / (ka * tau)) * np.ones((Nx - 1))
u_long_x = np.ones((Nx - 1))
d_long_x = np.ones((Nx - 1))

m_short1_x = -(2 + hx**2 / (ka * tau)) * np.ones((Mx - 1))
u_short1_x = np.ones((Mx - 1))
d_short1_x = np.ones((Mx - 1))

m_short2_x = -(2 + hx**2 / (ka * tau)) * np.ones((Nx - Kx - 1))
u_short2_x = np.ones((Nx - Kx - 1))
d_short2_x = np.ones((Nx - Kx - 1))

m_long_z = -(2 + hz**2 / (ka * tau)) * np.ones((Nz - 1))
u_long_z = np.ones((Nz - 1))
d_long_z = np.ones((Nz - 1))

m_short1_z = -(2 + hz**2 / (ka * tau)) * np.ones((Mz - 1))
u_short1_z = np.ones((Mz - 1))
d_short1_z = np.ones((Mz - 1))
m_short2_z = -(2 + hz**2 / (ka * tau)) * np.ones((Nz - Kz - 1))
u_short2_z = np.ones((Nz - Kz - 1))
d_short2_z = np.ones((Nz - Kz - 1))

fv = np.zeros((Nx + 1, Nz + 1))
for j in range(1, Nz):
    for i in range(1, Nx):
        fv[i, j] = f(i * hx, j * hz)

hx2, hz2 = hx**2, hz**2

c1_x = -hx2 / (ka * tau)
c2_x = hx2 / ka
c3_x = hx2 / hz2

c1_z = -hz2 / (ka * tau)
c2_z = hz2 / ka
c3_z = hz2 / hx2

err = 1e4
count = 0
while err > eps:
    count += 1
    u_old = deepcopy(u) # Iteration m
    u_half = deepcopy(u) # Iteration m + 0.5
    
    # X direction
    u_half[:, 0] = u0 * np.ones((Nx + 1))
    u_half[:, Nz] = u0 * np.ones((Nx + 1))
    for j in range(1, Nz):
        u_half[0, j] = u0
        u_half[Nx, j] = u0
        if z[j] < e or z[j] > g: # Line doesn't intersect hole
            Fx = [
                c1_x * u_old[i, j] - c2_x * fv[i, j] -
                c3_x * (u_old[i, j + 1] - 2 * u_old[i, j] + u_old[i, j - 1])
                for i in range(1, Nx)
            ]
            Fx[0] -= u0
            Fx[-1] -= u0
            u_half[1 : Nx, j] = tridiag(d_long_x, u_long_x, m_long_x, Fx)
        else: # Line intersect hole
            # Left
            Fx1 = [
                c1_x * u_old[i, j] - c2_x * fv[i, j] -
                c3_x * (u_old[i, j + 1] - 2 * u_old[i, j] + u_old[i, j - 1])
                for i in range(1, Mx)
            ]
            Fx1[0] -= u0
            Fx1[-1] -= u1
            
            # Right
            Fx2 = [
                c1_x * u_old[i, j] - c2_x * fv[i, j] -
                c3_x * (u_old[i, j + 1] - 2 * u_old[i, j] + u_old[i, j - 1])
                for i in range(Kx + 1, Nx)
            ]
            Fx2[0] -= u1
            Fx2[-1] -= u0
            u_half[1 : Mx, j] = tridiag(d_short1_x, u_short1_x, m_short1_x, Fx1)
            u_half[Kx + 1 : Nx,j] = tridiag(d_short2_x, u_short2_x, m_short2_x, Fx2)
            u_half[Mx : Kx + 1, j] = u1 * np.ones((Kx - Mx + 1))
    
    # Z direction
    u[0, :] = u0 * np.ones((Nz + 1))
    u[Nx, :] = u0 * np.ones((Nz + 1))
    for i in range(1, Nx):
        u[i, 0] = u0
        u[i, Nz] = u0
        if x[i] < c or x[i] > d:  # Line doesn't intersect hole
            Fz = [
                c1_z * u_half[i, j] - c2_z * fv[i, j] -
                c3_z * (u_half[i + 1, j] - 2 * u_half[i, j] + u_half[i - 1, j])
                for j in range(1, Nz)
            ]
            Fz[0] -= u0
            Fz[-1] -= u0
            u[i,1:Nz] = tridiag(d_long_z, u_long_z, m_long_z, Fz)
        else: # Line intersect hole
            # Bottom
            Fz1 = [
                c1_z * u_half[i, j] - c2_z * fv[i, j] -
                c3_z * (u_half[i + 1, j] - 2 * u_half[i, j] + u_half[i - 1, j])
                for j in range(1, Mz)]
            Fz1[0] -= u0
            Fz1[-1] -= u1

            # Top
            Fz2 = [
                c1_z * u_half[i, j] - c2_z * fv[i, j] -
                c3_z * (u_half[i + 1, j] - 2 * u_half[i, j] + u_half[i - 1, j])
                for j in range(Kz + 1, Nz)
            ]
            Fz2[0] -= u1
            Fz2[-1] -= u0
            u[i, 1 : Mz] = tridiag(d_short1_z, u_short1_z, m_short1_z, Fz1)
            u[i, Kz + 1 : Nz] = tridiag(d_short2_z, u_short2_z, m_short2_z, Fz2)
            u[i, Mz : Kz + 1] = u1 * np.ones((Kz - Mz + 1))

    err = np.sum((u_old - u)**2)

In [ ]:
%matplotlib widget

print(f'Iterations: {count}')

plt.imshow(np.transpose(u), extent=[x[0], x[Nx - 1], z[Nz - 1], z[0]], cmap=cm.plasma)
hole_rect = Rectangle((c, e), d - c, g - e,
                        edgecolor='black', facecolor='black')
plt.gca().add_patch(hole_rect)
plt.colorbar()
plt.show()